# Imports

In [ ]:
import os
from collections import Counter
import datetime

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3
import numpy as np
from tqdm.notebook import tqdm

### Define paths

In [ ]:
data_path = '/home/alexey/Education/Skoltech/HackLab/Sber/data'
sample_sumb_path = os.path.join(data_path, 'sample_submission.csv')
table_1_path = os.path.join(data_path, 'kaggle_tab_1345', 'tab_1_orders.csv')
table_2_dirs = sorted([dirname for dirname in os.listdir(data_path) if 'sbermarket_tab_2' in dirname])
store_products_path = os.path.join(data_path, 'store_products_by_sales.csv')
train_path = os.path.join(data_path, 'train.csv')

## Find pre-last orders

In [ ]:
sample_sumb = pd.read_csv(sample_sumb_path)

In [4]:
sample_sumb.head()

,Id,Predicted
0,51,3239211 10720024 8493844 1965540 5009002 95218...
1,65,11693356 13392267 2464515 2245175 4748570 1134...
2,766,12773054 3692749 12726751 10041624 11893515 23...
3,1132,8613934 3599065 575614 4821474 11609320 699047...
4,1578,289404 4328240 3129788 9390886 11166655 173077...


In [5]:
test_user_ids = list(sample_sumb['Id'].unique())
print(f'Number of users in test: {len(test_user_ids)}')

Number of users in test: 107068


In [15]:
table_1 = pd.read_csv(table_1_path)

In [ ]:
table_1_in_test = table_1[table_1['user_id'].isin(test_user_ids)]

In [ ]:
def pre_last(x):
    if len(x) > 1:
        return sorted(x)[-2]
    else:
        return x.iloc[-1]

pre_last_order_dict = dict(table_1_in_test.groupby('user_id', sort=False)['order_id']\
                           .apply(lambda x: pre_last(x)))

In [ ]:
len(pre_last_order_dict)

### Save to csv

In [ ]:
pre_last_df = pd.DataFrame({'user_id': list(pre_last_order_dict.keys()), 
                            'pre_last_order_id': list(pre_last_order_dict.values())})

In [ ]:
pre_last_df.to_csv('pre_last_orders.csv', index=None)

### Load from csv

In [6]:
pre_last_df = pd.read_csv('pre_last_orders.csv')

In [7]:
pre_last_df.head()

,user_id,pre_last_order_id
0,187,15952443
1,400,13176781
2,576,16799033
3,608,16955655
4,766,17116500


In [8]:
user_to_order = dict(zip(pre_last_df['user_id'], pre_last_df['pre_last_order_id']))

In [9]:
pre_last_orders = set(pre_last_df['pre_last_order_id'])

In [10]:
len(pre_last_orders)

104238

In [11]:
2152537 in pre_last_orders

False

## Find pre-last products wherever it is possible

In [12]:
pre_last_orders_prods = dict()

for table_2_dir in table_2_dirs:
    full_dir_path = os.path.join(data_path, table_2_dir)
    filenames = sorted(list(os.listdir(full_dir_path)))
    
    for filename in filenames:
        full_filename = os.path.join(full_dir_path, filename)

        tab_2 = pd.read_csv(full_filename, usecols=['order_id', 'product_id'])
        tab_labeled = tab_2.groupby(by='order_id').product_id.apply(list)
        tab_labeled_orders = set(list(tab_labeled.index)) & pre_last_orders
        print(f'{len(tab_labeled_orders)} found')
        if len(tab_labeled_orders) > 0:
            pre_last_orders_prods.update(dict(tab_labeled[list(tab_labeled_orders)]))

15 found
17 found
31 found
58 found
50 found
79 found
87 found
154 found
164 found
195 found
277 found
420 found
546 found
696 found
884 found
1198 found
1262 found
1292 found
1379 found
2040 found
2657 found
3609 found
4718 found
8007 found
13550 found
23129 found


### We need to find each order retailer or no retailer

In [13]:
order_retailer_dict = dict(zip(pre_last_orders, [None for _ in range(len(pre_last_orders))]))

Update it with the 1-st table

In [16]:
table_1_ord_ret = table_1[['order_id', 'retailer']].drop_duplicates()

In [17]:
table_1_ord_ret = table_1_ord_ret[table_1_ord_ret['order_id'].isin(pre_last_orders)]

In [18]:
table_1_ord_ret = table_1_ord_ret.reset_index(drop=True)

In [19]:
table_1_ord_ret.head()

,order_id,retailer
0,15952443,ВкусВилл
1,13176781,METRO
2,16799033,SELGROS
3,16955655,Ашан
4,17116500,METRO


In [20]:
len(table_1_ord_ret['retailer'].unique())

34

In [21]:
for i in tqdm(range(len(table_1_ord_ret))):
    order_retailer_dict[table_1_ord_ret.iloc[i, 0]] = table_1_ord_ret.iloc[i, 1]

## Finally, for each test user fill his products

In [45]:
answer_dict = dict()

### 1. Fill with known order info

In [46]:
for test_user_id in tqdm(user_to_order):
    user_order = user_to_order[test_user_id]
    if user_order in pre_last_orders_prods:
        answer_dict[test_user_id] = pre_last_orders_prods[user_order][:50]

### 2. For not filled let's fill with most popular

In [47]:
store_products_df = pd.read_csv(store_products_path)

In [48]:
store_to_prods = dict(store_products_df.groupby('retailer')['product_id'].apply(list))

In [49]:
for test_user_id in tqdm(answer_dict):
    current_prod_list = answer_dict[test_user_id]
    if len(current_prod_list) < 50:
        user_order = user_to_order[test_user_id]
        user_shop = order_retailer_dict[user_order]
        popular_shop_prods = store_to_prods[user_shop]
        
        num_filled = len(current_prod_list)
        answer_dict[test_user_id].extend(popular_shop_prods[:(50-num_filled)])

In [50]:
for test_user_id in tqdm(test_user_ids):
    if test_user_id in answer_dict:
        assert len(answer_dict[test_user_id]) == 50, f'strange at: {test_user_id}'

### 3. Fill with most popular by shop for missed ids

In [51]:
most_popular_prods = list(store_products_df.sort_values(by='sales', ascending=False)['product_id']\
                          .unique()[:50])

In [52]:
for test_user_id in tqdm(test_user_ids):
    if test_user_id not in answer_dict:
        if test_user_id in user_to_order:
            user_order = user_to_order[test_user_id]
            user_shop = order_retailer_dict[user_order]
            if user_shop in store_to_prods:
                popular_shop_prods = store_to_prods[user_shop]
                if len(popular_shop_prods) >= 50:
                    answer_dict[test_user_id] = popular_shop_prods[:50]
                else:
                    answer_dict[test_user_id] = popular_shop_prods
                    num_filled = len(popular_shop_prods)
                    answer_dict[test_user_id].extend(most_popular_prods[:(50 - num_filled)])

In [53]:
for test_user_id in tqdm(test_user_ids):
    if test_user_id in answer_dict:
        assert len(answer_dict[test_user_id]) == 50, f'strange at: {test_user_id}'

### 4. Finally, predict for people with no data

In [54]:
for test_user_id in tqdm(test_user_ids):
    if test_user_id not in answer_dict:
        answer_dict[test_user_id] = most_popular_prods

In [55]:
for test_user_id in tqdm(test_user_ids):
    if test_user_id in answer_dict:
        assert len(answer_dict[test_user_id]) == 50, f'strange at: {test_user_id}'

## Write results

In [61]:
def list_to_str(some_list):
    return ' '.join([str(elem) for elem in some_list])

In [62]:
list_to_str([1,2,3])

'1 2 3'

In [63]:
sample_sumb.head()

,Id,Predicted
0,51,3239211 10720024 8493844 1965540 5009002 95218...
1,65,11693356 13392267 2464515 2245175 4748570 1134...
2,766,12773054 3692749 12726751 10041624 11893515 23...
3,1132,8613934 3599065 575614 4821474 11609320 699047...
4,1578,289404 4328240 3129788 9390886 11166655 173077...


In [64]:
sample_sumb.columns

Index(['Id', 'Predicted'], dtype='object')

In [65]:
dict_for_df = {'Id': [], 
               'Predicted': []}

In [70]:
for subm_id in sample_sumb['Id']:
    dict_for_df['Id'].append(subm_id)
    dict_for_df['Predicted'].append(list_to_str(answer_dict[subm_id]))

In [71]:
killer_submit = pd.DataFrame(dict_for_df)

In [73]:
killer_submit.to_csv('killer_submit.csv', index=None)